# Esercizio 4 - Segmentation

## Approccio

L'algoritmo utilizza una tecnica *brute force* per trovare i tagli corretti. Prova diverse combinazioni di tagli generandoli in maniera randomica e associa ad ognuna un punteggio. Il punteggio è calcolato come la somma degli *n* termini più frequnti all'interno di ogni segmento.

L'idea alla base è che per ogni argomento ci siano un numero di parole che occorrono molte volte, e quindi l'algoritmo andrà a cercare 
i tagli che vanno a massimizzare questi valori, e che quindi vanno ad isolare i termini più frequenti per ogni segmento.

L'algoritmo ha bisogno del numero di tagli in input.

## Datasets

*data/segmentation_eng*: Il dataset è costituito da pezzi dei paragrafi di wikipedia di 4 argomenti diversi, in questo caso i 4 argomenti sono:
  - Gorilla
  - Quantum Computing
  - Astronomy
  - Kendrick Lamar Biography

I tagli corretti sono alla riga 59/60, alla riga 102/103 e alla riga 181/182.


### Imports

In [5]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import random
import numpy as np

### Funzioni

In [6]:
def remove_stop_words_ita(phrase):
    stop_words = stopwords.words('italian')
    phrase = phrase.split()
    phrase = [word for word in phrase if word not in stop_words]
    return phrase

def get_text_from_file(path):
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def get_score(text, n_most_words):
    '''
    Calcolo le n_most_words più frequenti e 
    restituisco la somma delle loro frequenze 
    come score del segmento di testo 
    '''
    score = 0
    c = Counter()
    most_common = []
    for row in text:
        c.update(row)
        
    most_common = c.most_common(n_most_words)
    
    for el in most_common:
        score = score + el[1]
        
    return score

def extract_segment(file, start, end):
    segment = []
    for i in range(int(start), int(end)):
        segment.append(file[i])
    return segment

def get_cuts(ntopic, num_lines):
    cut = np.zeros(ntopic)

    for k in range(1, ntopic):
        cut[k] = random.randint(1, num_lines-1)
    
    cut = np.append(cut, num_lines)

    return sorted(cut)

def load_file(file_path):
    file = get_text_from_file(file_path)
    c = Counter()
    num_lines = sum(1 for line in open(file_path)) # Number of lines in the file

    for row in file:
        c.update(row)
        
    return num_lines, file, c
    

### Algorithm

In [7]:
def max_scores(file, ntopic, n_most_words, n_iteration, num_lines): 
    scores = np.zeros(ntopic)
    sum_scores, max = 0, 0
    max_cut = []
    
    for f in range(n_iteration): 

        cut = get_cuts(ntopic, num_lines)
        
        for i in range(len(cut)-1):
            text = extract_segment(file, cut[i], cut[i+1])
            scores[i] = get_score(text, n_most_words)
        
        sum_scores = sum(scores)
        if(sum_scores > max):
            max = sum_scores
            max_cut = cut
            
    return max_cut, max

### Data Processing

In [8]:
n_most_words = 3
n_topics = 4
n_iteration = 10000 
c = Counter()
file = '../data/segmentation_eng.txt'

file_data = load_file(file)

num_lines = file_data[0]
file = file_data[1]

for row in file:
    c.update(row)

res = max_scores(file, n_topics, n_most_words, n_iteration, num_lines)

print(f'\nBest cut at lines -->  {res[0]}\nthe max sum is    -->  {res[1]}')


Best cut at lines -->  [0.0, 54.0, 102.0, 180.0, 254.0]
the max sum is    -->  361.0


### Analisi dei risultati e limiti di questo approccio

Problemi dell'approccio utilizzato:

- è poco **scalabile** sulla *dimensione del testo*, a cusa dello sforzo computazionale che cresce con il numero di righe del file

- la metrica di **similarity** può essere migliorata, impiegando ad esempio una cosine similarity, dopo aver estratto dei vettori numerici dal testo

- con un **dataset** con meno varietà negli argomenti le prestazioni potrebbero calare drasticamente
